In [1]:
import nltk
import random
from nltk.corpus import movie_reviews,stopwords
from nltk.classify.scikitlearn import SklearnClassifier
from nltk.classify import ClassifierI

from statistics import mode

import pickle

from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

# nltk.download('movie_reviews')


stop_words = set(stopwords.words("English"))
# adding unnecessary signs and symbols
for i in range(0,256):
    stop_words.add(chr(i))

# category( pos/neg )
documents = []

# pickle this shit
for category in movie_reviews.categories():
    for fileid in movie_reviews.fileids(category):
        word_list = list(movie_reviews.words(fileid))
        check_list = []
        for word in word_list:
            if word not in stop_words and not word.isdigit():
                check_list.append(word)
        documents.append((check_list,category))

# print(documents[0])

random.shuffle(documents)

In [2]:
all_words = []

for words in movie_reviews.words():
	all_words.append(words.lower())

all_words = nltk.FreqDist(all_words)		# most_common -> least_common

# print(all_words.most_common(15))
print(all_words["stop"])

words_features = list(all_words.keys())[:5000]

268


In [3]:
def find_features(docum):
	words = set(docum)
	features = {}
	for word in words_features:
		features[word] = word in words
	return features

# print(find_features(movie_reviews.words('neg/cv000_29416.txt')))
feature_sets = [(find_features(rev),category) for (rev,category) in documents]

print(len(feature_sets))

training_set = feature_sets[:1900]    
testing_set  = feature_sets[1900:]  

# Naive Bayes 
# print(training_set[:4])
# posterior = (prior_occurrences x likelihood) / evidence

2000


In [4]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

# pickling
# classifier_f = open('naiveBayes.pickle','rb')
# classifier = pickle.load(classifier_f)
# classifier_f.close()

print("Original NB accuracy: ",nltk.classify.accuracy(classifier,testing_set)*100)
classifier.show_most_informative_features(15)    # top 15 

# save_classifier = open("naiveBayes.pickle",'wb')
# pickle.dump(classifier,save_classifier)
# save_classifier.close()

Original NB accuracy:  83.0
Most Informative Features
               ludicrous = True              neg : pos    =     14.6 : 1.0
               atrocious = True              neg : pos    =     11.0 : 1.0
                    hers = True              pos : neg    =     10.4 : 1.0
                   sucks = True              neg : pos    =     10.2 : 1.0
                 idiotic = True              neg : pos    =      9.3 : 1.0
              accessible = True              pos : neg    =      9.0 : 1.0
                  annual = True              pos : neg    =      9.0 : 1.0
               stupidity = True              neg : pos    =      9.0 : 1.0
                 frances = True              pos : neg    =      8.4 : 1.0
           unimaginative = True              neg : pos    =      8.3 : 1.0
                  justin = True              neg : pos    =      8.3 : 1.0
                 wasting = True              neg : pos    =      7.7 : 1.0
                narrates = True              p

In [5]:
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)

# print(testing_set[0])
print("test classification: ",str(MNB_classifier.classify(testing_set[0][0])))
print("test label :",testing_set[0][1])

#GNB_classifier = SklearnClassifier(GaussianNB())
#GNB_classifier.train(training_set)
#print("Gaussian NB accuracy: ",nltk.classify.accuracy(GNB_classifier,testing_set)*100)

BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)

LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(training_set)
print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGDClassifier_classifier, testing_set))*100)

# SVC_classifier = SklearnClassifier(SVC())
# SVC_classifier.train(training_set)
# print("SVC_classifier accuracy percent:", (nltk.classify.accuracy(SVC_classifier, testing_set))*100)

LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(training_set)
print("NuSVC_classifier accuracy percent:", (nltk.classify.accuracy(NuSVC_classifier, testing_set))*100)

# tweak these to get + - 10% accuracy difference 
# Note : redundancy in data sets consideration ( most common words may contain punctuations)

MNB_classifier accuracy percent: 84.0
test classification:  pos
test label : pos
BernoulliNB_classifier accuracy percent: 84.0
LogisticRegression_classifier accuracy percent: 87.0


/usr/local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier_classifier accuracy percent: 89.0
LinearSVC_classifier accuracy percent: 86.0
NuSVC_classifier accuracy percent: 90.0


In [6]:
class VoteClassifier(ClassifierI):
	def __init__(self,*classifiers):
		self.classifiers = classifiers
	
	def classify(self,features):
		votes = []
		for cls in self.classifiers:
			v = cls.classify(features)
			votes.append(v)
		return mode(votes)
	
	def confidence(self,features):
		votes = []
		for cls in self.classifiers:
			v = cls.classify(features)
			votes.append(v)
		choice = votes.count(mode(votes))
		confid = choice/len(votes)
		return confid 


voted_classifier = VoteClassifier(classifier,
								MNB_classifier,
								BernoulliNB_classifier,
								LogisticRegression_classifier,
								SGDClassifier_classifier,
								LinearSVC_classifier,
								NuSVC_classifier)

print("voted_classifier accuracy percent:", (nltk.classify.accuracy(voted_classifier, testing_set))*100)

print("Labeled test1 as: ",testing_set[0][1])
print("Classification test1 : ",voted_classifier.classify(testing_set[0][0]))
print("Confidence : ",voted_classifier.confidence(testing_set[0][0]))

print("Labeled test2 as: ",testing_set[1][1])
print("Classification test2 : ",voted_classifier.classify(testing_set[1][0]))
print("Confidence : ",voted_classifier.confidence(testing_set[1][0]))

print("Labeled test3 as: ",testing_set[2][1])
print("Classification test3 : ",voted_classifier.classify(testing_set[2][0]))
print("Confidence : ",voted_classifier.confidence(testing_set[2][0]))

print("Labeled test4 as: ",testing_set[3][1])
print("Classification test4 : ",voted_classifier.classify(testing_set[3][0]))
print("Confidence : ",voted_classifier.confidence(testing_set[3][0]))

voted_classifier accuracy percent: 89.0
Labeled test1 as:  pos
Classification test1 :  pos
Confidence :  1.0
Labeled test2 as:  pos
Classification test2 :  pos
Confidence :  1.0
Labeled test3 as:  pos
Classification test3 :  pos
Confidence :  1.0
Labeled test4 as:  neg
Classification test4 :  neg
Confidence :  1.0
